# Preprocesamiento de los datos - Análisis de suelos


## Importaciones

In [16]:
import numpy as np
import pandas as pd

## Funciones

In [33]:
def clean_float_values(value,rares):
    '''Limpia valores en columnas float
    
    Procesa valores string realizando las siguientes acciones:
      - Reemplaza ',' por '.'
      - Reemplaza '..' por '.'
      - Elimina el signo '>' dejando el límite superior como valor actual
      - Reemplaza los límites inferiores por la mitad de su valor
      - Reemplaza valores inválidos por NaN
    '''
    if type(value) != str:
        return value
    
    value = value.replace(',', '.')
    value = value.replace('..', '.')
    value = value.replace('>', '')
    
    if value[0] == '<':
        value = float((value[1:]).strip()) / 2        
    else:
        try:
            float(value)
        except Exception as e:
            if value in rares.keys():
                rares[value] += 1
            else:
                rares[value] = 1
            value = 'NaN'
    return value

In [18]:
def clean_string_values(value):
    value = value.replace(' ', '_')
    value = value.replace('__', '_')
    return value

## Dataset: Análisis de suelos

### Carga de los datos

In [19]:
df_analisis = pd.read_csv('suelos_original.csv',dtype=str)

### Tratamiento

#### Renombrar columnas

In [20]:
oDict = {
    'numfila': 'id',
    'Departamento': 'departamento',
    'Municipio': 'municipio',
    'Cultivo': 'cultivo',
    'Estado': 'estado',
    'Tiempo Establecimiento': 'tiempo_establecimiento',
    'Topografia': 'topografia',
    'Drenaje': 'drenaje',
    'Riego': 'riego',
    'Fertilizantes aplicados': 'fertilizantes',
    'FechaAnalisis': 'fecha',
    'pH agua:suelo 2,5:1,0': 'ph',
    'Materia orgánica (MO) %': 'materia_organica',
    'Fósforo (P) Bray II mg/kg': 'fosforo',
    'Azufre (S) Fosfato monocalcico mg/kg': 'azufre',
    'Acidez (Al+H) KCL cmol(+)/kg': 'acidez',
    'Aluminio (Al) intercambiable cmol(+)/kg': 'aluminio',
    'Calcio (Ca) intercambiable cmol(+)/kg': 'calcio',
    'Magnesio (Mg) intercambiable cmol(+)/kg': 'magnesio',
    'Potasio (K) intercambiable cmol(+)/kg': 'potasio',
    'Sodio (Na) intercambiable cmol(+)/kg': 'sodio',
    'capacidad de intercambio cationico (CICE) suma de bases cmol(+)/kg': 'cice',
    'Conductividad el‚ctrica (CE) relacion 2,5:1,0 dS/m': 'ce',
    'Hierro (Fe) disponible olsen mg/kg': 'hierro_olsen',
    'Cobre (Cu) disponible mg/kg': 'cobre',
    'Manganeso (Mn) disponible Olsen mg/kg': 'manganeso',
    'Zinc (Zn) disponible Olsen mg/kg': 'zinc_olsen',
    'Boro (B) disponible mg/kg': 'boro',
    'Hierro (Fe) disponible doble \xa0cido mg/kg': 'hierro_doble_acido',
    'Cobre (Cu) disponible doble acido mg/kg': 'cobre_doble_acido',
    'Manganeso (Mn) disponible doble acido mg/kg': 'manganeso_doble_acido',
    'Zinc (Zn) disponible doble \xa0cido mg/kg': 'zinc_doble_acido', 
    'Secuencial': 'secuencial'
}
df_analisis.rename(columns=oDict, inplace=True)

#### Eliminar columnas sin valores

In [21]:
df_analisis.drop(columns=['secuencial', 'fecha'], inplace=True)

#### Formato de valores en columnas

In [25]:
# Cambiar los valores string a minúsculas
columnas = [
    'departamento', 'municipio', 'cultivo', 'estado', 'tiempo_establecimiento', 
    'topografia', 'drenaje', 'riego', 'fertilizantes'
]
df_analisis[columnas] = df_analisis[columnas].applymap(str.lower)
#for columna in columnas:
#    df_analisis[columna] = df_analisis[columna].str.lower()

#### Corrección de valores float

In [37]:
rares = {}

# Limpieza de carácteres no numéricos y conversión de valores numéricos a float
columnas = [
    'ph', 'fosforo', 'azufre', 'acidez', 'aluminio', 
    'calcio', 'magnesio', 'potasio', 'sodio', 'ce', 'hierro_olsen', 
    'cobre', 'manganeso', 'zinc_olsen', 'boro', 'hierro_doble_acido',
    'cobre_doble_acido', 'manganeso_doble_acido', 'zinc_doble_acido'
]

df_analisis[columnas] = df_analisis[columnas].applymap(clean_float_values,rares=rares)

if len(rares):
    print("Valores problemáticos convertidos a NaN:\n", rares)

#### Corrección de valores string

In [38]:
df_analisis["topografia"] = df_analisis["topografia"].str.replace(
    "error: #n/a", "no indica")

oDict = {
    'error: #n/a': "no indica",
    'buen drenaje': 'bueno',
    'regular drenaje': 'regular',
    'mal drenaje': 'malo'
}
df_analisis["drenaje"].replace(oDict, inplace=True)

In [43]:
columnas = [
    'estado', 
    'tiempo_establecimiento', 
    'topografia', 
    'drenaje', 
    'riego'
]

df_analisis[columnas] = df_analisis[columnas].applymap(clean_string_values)

---

#### Eliminación de registros


Esta operación es discutible ya que esos registros pueden servir para la detección de anomalías aunque no sirvan para el modelo de predicción
```python
indices = df_analisis[
    (df_analisis["cultivo"] == "no indica") & 
    (df_analisis["estado"] == "no indica") & 
    (df_analisis["tiempo_establecimiento"] == "no indica")].index
df_analisis.drop(index=indices, axis=0, inplace=True)
```

In [44]:
# Eliminar las filas con el valor "borrar" en la columna municipio
df_analisis = df_analisis[df_analisis['municipio'] != 'borrar']

#### Columnas categorizadas

In [45]:
# Crear columnas dummy a partir de la columna topografia
columnas = [
    'estado', 'tiempo_establecimiento', 'topografia', 'drenaje', 'riego']

for columna in columnas:
    df_analisis = pd.get_dummies(df_analisis, columns=[columna])

# Ajustar la dummificación de las columnas compuestas
columnas = {
    'topografia_ondulado_y_pendiente' : ['topografia_ondulado', 'topografia_pendiente'],
    'topografia_plano_y_ondulado' : ['topografia_plano', 'topografia_ondulado'],
    'topografia_plano_y_pendiente' : ['topografia_plano', 'topografia_pendiente'],
    'riego_aspersión_y_goteo': ['riego_aspersión', 'riego_goteo'],
    'riego_aspersión_y_gravedad': ['riego_aspersión', 'riego_gravedad'],
    'riego_goteo_y_gravedad': ['riego_goteo', 'riego_gravedad'],
}
for key, value in columnas.items():
    indices = df_analisis[(df_analisis[key] == 1)].index
    df_analisis.loc[indices, value[0]] = 1
    df_analisis.loc[indices, value[1]] = 1
    df_analisis.drop(columns=[key], inplace=True)

### Exportar conjunto de datos

In [16]:
df_analisis.to_csv('suelos_preprocesado.csv', index=False)

---
Fin del notebook